In [1]:
import pandas as pd 
import json
import re
import psycopg2
import nltk
import math
import uuid
import psycopg2.extensions
import psycopg2.extras
from psycopg2 import sql
from nltk.corpus import stopwords
from configparser import ConfigParser
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
stopwords = set(stopwords.words('english'))

In [2]:
def connect():
    try:
        
        parser = ConfigParser()
        parser.read('../../database.ini')

        #params = config()
        host = parser.get('postgresql','DB_HOST')
        port = parser.get('postgresql','DB_PORT')
        database =parser.get('postgresql','DB_NAME')
        user = parser.get('postgresql','DB_USER')
        password = parser.get('postgresql','DB_PW')

        #params = config()
        #print(host, port, password, user, database)
        conn = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)

        curr = conn.cursor()
        
#         query = """SELECT table_name from information_schema.tables WHERE table_schema = 'public'"""
#         curr.execute(query)
        
#         rows = curr.fetchall()

#         for row in rows:
#             print(row[0])

        
    except( Exception, psycopg2.DatabaseError) as err:
        print(err)
    finally:
        return curr, conn
    
cur, con = connect()

In [3]:
def clean_data(text):
    if text == '' or not list(text) or text == {} or text == '{}':
        return math.nan
    # text = json.loads(text)
    final = [(x +"-" + y).lower() for x,y in zip(list(text), list(text.values()))]
    return "  ".join(final)

In [4]:
#get wishlist products 
query = "SELECT * FROM watchlist_cpu"
cpus = pd.read_sql_query(query, con)
cpus = pd.DataFrame(cpus)

query = "SELECT * FROM watchlist_gpu"
gpus = pd.read_sql_query(query, con)

wishlist_products = cpus.append(gpus)
#any preprocessing needing to be done
#table = remove_columns_from_products(table)
wishlist_products.drop_duplicates(subset='product_id', keep=False, inplace=True)

wishlist_products


,user_id,product_id
0,c8a93f30-fe1b-4cea-a7e2-49c37e908133,2d26b339-7f4c-485e-a9d2-0050dfe2f78e
1,c8a93f30-fe1b-4cea-a7e2-49c37e908133,04cd4be7-b116-4a05-8a70-99402fd984f8
2,c8a93f30-fe1b-4cea-a7e2-49c37e908133,1ecc4407-702d-4881-98d3-707231ea1bd5
3,c8a93f30-fe1b-4cea-a7e2-49c37e908133,2320d856-c9f8-4367-9ceb-fc25ec2abc48
0,c8a93f30-fe1b-4cea-a7e2-49c37e908133,6c06f47c-c4ee-4cd3-88a9-bb55890aa923
1,c8a93f30-fe1b-4cea-a7e2-49c37e908133,34748fb8-6c65-4bfe-9d8f-3d8749b18cbf
2,c8a93f30-fe1b-4cea-a7e2-49c37e908133,79cc0dc7-383f-43fe-8a9d-406012f6edb7
3,c8a93f30-fe1b-4cea-a7e2-49c37e908133,909f4df4-dd36-46fb-bc39-c5208598b5ec
4,c8a93f30-fe1b-4cea-a7e2-49c37e908133,b7dc3042-0330-42d9-bebb-59601adfb987


In [5]:
#get all products
query = "SELECT id, brand, retailer, type, description FROM cpus"
cpus = pd.read_sql_query(query, con)
cpus = pd.DataFrame(cpus)

query = "SELECT id, brand, retailer, type, description FROM gpus"
gpus = pd.read_sql_query(query, con)

all_products = cpus.append(gpus)

# any preprocessing needed to be done
all_products['description_cl'] = all_products['description'].apply(clean_data)
all_products.drop(['description'], axis=1, inplace=True)
all_products.dropna(inplace=True)

all_products.head()

,id,brand,retailer,type,description_cl
0,04cd4be7-b116-4a05-8a70-99402fd984f8,AMD,Evetech,cpu,system memory type-ddr4 pci express® version-...
2,1a4c23c6-53d9-4728-bc8e-1723a5d0511e,AMD,Evetech,cpu,system memory type-ddr4 pci express® version-...
6,1653365c-ffb9-4b8f-b96a-42a5b4817dde,AMD,Evetech,cpu,system memory type-ddr4 pci express® version-...
15,00a27c9c-8985-4fed-8496-3eab23a303c9,AMD,Evetech,cpu,memory channels-2 system memory type-ddr4 pc...
19,d0b620a6-8078-4f71-a95f-1bdc7fae7d91,AMD,Evetech,cpu,system memory type-ddr4 pci express® version-...


In [6]:
all_products.rename(columns={'id': 'product_id'}, inplace=True)
# all_products['all_features'] = all_products['brand'] +" "+ all_products['retailer'] +" "+ all_products['description_cl']
# all_products['all_features'] = all_products['all_features'].apply(lambda x: x.lower())
products_cpu = all_products[all_products['type'] == 'cpu']
products_gpu = all_products[all_products['type'] == 'gpu']

vectorizor = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0)
tfidf_matrix_cpu = vectorizor.fit_transform(products_cpu['description_cl'])
tfidf_matrix_gpu = vectorizor.fit_transform(products_gpu['description_cl'])


similarity_cpu = linear_kernel(tfidf_matrix_cpu, tfidf_matrix_cpu)
similarity_gpu = linear_kernel(tfidf_matrix_gpu, tfidf_matrix_gpu)

rec_cpu = pd.Series(products_cpu['product_id'])
rec_gpu = pd.Series(products_gpu['product_id'])

In [7]:
products_gpu

,product_id,brand,retailer,type,description_cl
2,6c06f47c-c4ee-4cd3-88a9-bb55890aa923,ASUS ROG Strix,Evetech,gpu,boost clock-1.67 ghz memory size-12 gb memor...
3,34748fb8-6c65-4bfe-9d8f-3d8749b18cbf,MSI,Evetech,gpu,boost clock-1.67 ghz memory size-12 gb memor...
4,79cc0dc7-383f-43fe-8a9d-406012f6edb7,ASUS TUF Gaming,Evetech,gpu,boost clock-1.67 ghz memory size-12 gb memor...
5,04eff54f-09c1-4f43-b72b-a86392820762,ASUS ROG Strix LC,Evetech,gpu,rops-128 compute units-80 texture units-320 ...
6,d00c8bb0-5db9-4caa-91a2-c9963eee8a3f,ASUS TUF,Evetech,gpu,boost clock-1.67 ghz memory size-12 gb memor...
...,...,...,...,...,...
115,9029a238-8de7-4a01-9307-0bdd2ced6656,Gigabyte,Dreamware,gpu,rops-64 compute units-32 texture units-128 ...
116,fcdbc3d9-6c8e-4345-a40d-fc1394ab233a,Gigabyte,Dreamware,gpu,rops-64 compute units-40 texture units-160 ...
117,22242759-13da-4662-82eb-27a59b2d04d9,PNY Nvidia,Dreamware,gpu,rtx-ops-84t vr ready-yes rays cast-10 giga r...
118,e94628d2-ff36-4263-badb-fe0e02969f68,NVIDIA PNY,Dreamware,gpu,rtx-ops-62t vr ready-yes rays cast-8 giga ra...


In [8]:
products_cpu

,product_id,brand,retailer,type,description_cl
0,04cd4be7-b116-4a05-8a70-99402fd984f8,AMD,Evetech,cpu,system memory type-ddr4 pci express® version-...
2,1a4c23c6-53d9-4728-bc8e-1723a5d0511e,AMD,Evetech,cpu,system memory type-ddr4 pci express® version-...
6,1653365c-ffb9-4b8f-b96a-42a5b4817dde,AMD,Evetech,cpu,system memory type-ddr4 pci express® version-...
15,00a27c9c-8985-4fed-8496-3eab23a303c9,AMD,Evetech,cpu,memory channels-2 system memory type-ddr4 pc...
19,d0b620a6-8078-4f71-a95f-1bdc7fae7d91,AMD,Evetech,cpu,system memory type-ddr4 pci express® version-...
24,1a811387-923e-42d3-9e18-16c3b9ef3202,AMD,Evetech,cpu,memory channels-2 system memory type-ddr4 pc...
34,18b0243a-ff92-47b0-bbbd-eef96f55ccee,AMD,Amptek,cpu,system memory type-ddr4 pci express® version-...
37,b4728b3c-440f-4fb9-bd88-510732e5ad67,AMD,Amptek,cpu,memory channels-2 system memory type-ddr4 pc...
41,b8a06283-c102-4e7b-becf-fd7ddb587209,AMD,Amptek,cpu,system memory type-ddr4 pci express® version-...
59,5b43761f-898d-4870-aee0-4790a8b88996,AMD,Amptek,cpu,system memory type-ddr4 pci express® version-...


In [9]:
all_products.head()

,product_id,brand,retailer,type,description_cl
0,04cd4be7-b116-4a05-8a70-99402fd984f8,AMD,Evetech,cpu,system memory type-ddr4 pci express® version-...
2,1a4c23c6-53d9-4728-bc8e-1723a5d0511e,AMD,Evetech,cpu,system memory type-ddr4 pci express® version-...
6,1653365c-ffb9-4b8f-b96a-42a5b4817dde,AMD,Evetech,cpu,system memory type-ddr4 pci express® version-...
15,00a27c9c-8985-4fed-8496-3eab23a303c9,AMD,Evetech,cpu,memory channels-2 system memory type-ddr4 pc...
19,d0b620a6-8078-4f71-a95f-1bdc7fae7d91,AMD,Evetech,cpu,system memory type-ddr4 pci express® version-...


In [10]:
temp = []
all_products.set_index('product_id', inplace=True)
products_cpu.set_index('product_id', inplace=True)
products_gpu.set_index('product_id', inplace=True)
wishlist_products.reset_index()

,index,user_id,product_id
0,0,c8a93f30-fe1b-4cea-a7e2-49c37e908133,2d26b339-7f4c-485e-a9d2-0050dfe2f78e
1,1,c8a93f30-fe1b-4cea-a7e2-49c37e908133,04cd4be7-b116-4a05-8a70-99402fd984f8
2,2,c8a93f30-fe1b-4cea-a7e2-49c37e908133,1ecc4407-702d-4881-98d3-707231ea1bd5
3,3,c8a93f30-fe1b-4cea-a7e2-49c37e908133,2320d856-c9f8-4367-9ceb-fc25ec2abc48
4,0,c8a93f30-fe1b-4cea-a7e2-49c37e908133,6c06f47c-c4ee-4cd3-88a9-bb55890aa923
5,1,c8a93f30-fe1b-4cea-a7e2-49c37e908133,34748fb8-6c65-4bfe-9d8f-3d8749b18cbf
6,2,c8a93f30-fe1b-4cea-a7e2-49c37e908133,79cc0dc7-383f-43fe-8a9d-406012f6edb7
7,3,c8a93f30-fe1b-4cea-a7e2-49c37e908133,909f4df4-dd36-46fb-bc39-c5208598b5ec
8,4,c8a93f30-fe1b-4cea-a7e2-49c37e908133,b7dc3042-0330-42d9-bebb-59601adfb987


In [11]:
all_products.head()

,brand,retailer,type,description_cl
product_id,,,,
04cd4be7-b116-4a05-8a70-99402fd984f8,AMD,Evetech,cpu,system memory type-ddr4 pci express® version-...
1a4c23c6-53d9-4728-bc8e-1723a5d0511e,AMD,Evetech,cpu,system memory type-ddr4 pci express® version-...
1653365c-ffb9-4b8f-b96a-42a5b4817dde,AMD,Evetech,cpu,system memory type-ddr4 pci express® version-...
00a27c9c-8985-4fed-8496-3eab23a303c9,AMD,Evetech,cpu,memory channels-2 system memory type-ddr4 pc...
d0b620a6-8078-4f71-a95f-1bdc7fae7d91,AMD,Evetech,cpu,system memory type-ddr4 pci express® version-...


In [12]:
for i, pro in all_products.iterrows():
    for j, wish in wishlist_products.iterrows():
        if i == wish['product_id']:
            print(i, wish['product_id'])
            temp.append((wish['product_id'], pro['type']))

04cd4be7-b116-4a05-8a70-99402fd984f8 04cd4be7-b116-4a05-8a70-99402fd984f8
6c06f47c-c4ee-4cd3-88a9-bb55890aa923 6c06f47c-c4ee-4cd3-88a9-bb55890aa923
34748fb8-6c65-4bfe-9d8f-3d8749b18cbf 34748fb8-6c65-4bfe-9d8f-3d8749b18cbf
79cc0dc7-383f-43fe-8a9d-406012f6edb7 79cc0dc7-383f-43fe-8a9d-406012f6edb7
909f4df4-dd36-46fb-bc39-c5208598b5ec 909f4df4-dd36-46fb-bc39-c5208598b5ec
b7dc3042-0330-42d9-bebb-59601adfb987 b7dc3042-0330-42d9-bebb-59601adfb987


In [27]:
types = pd.DataFrame(temp, columns = ['product_id', 'type'])
wishlist_cpu = types[types['type'] == 'cpu']#wishlist_description[wishlist_description['product_id'] == types['product_id'].item() & types['type'].item() == 'cpu']
wishlist_gpu = types[types['type'] == 'gpu']#wishlist_description[wishlist_description['type'] == 'gpu']

wishlist_gpu

,product_id,type
1,6c06f47c-c4ee-4cd3-88a9-bb55890aa923,gpu
2,34748fb8-6c65-4bfe-9d8f-3d8749b18cbf,gpu
3,79cc0dc7-383f-43fe-8a9d-406012f6edb7,gpu
4,909f4df4-dd36-46fb-bc39-c5208598b5ec,gpu
5,b7dc3042-0330-42d9-bebb-59601adfb987,gpu


In [28]:
wishlist_cpu

,product_id,type
0,04cd4be7-b116-4a05-8a70-99402fd984f8,cpu


In [31]:
def recommend(idx, similarity, types):
    
    #find the index of the product in the index series
    if types == 'cpu':
        item = rec_cpu[rec_cpu == idx].index[0]
    else: 
        item = rec_gpu[rec_gpu == idx].index[0]
        
    #get array of similarities and sort them in descending order
    similarities = list(enumerate(similarity[item]))
    similarities = sorted(similarities, key=lambda x: x[1], reverse=True)
    
    #get the top 5 products, excluding the product itself
    similarities = similarities[0:6]
   
    #get index of top 5 
    index = [i[0] for i in similarities]
    
    #get the products using the the retrieved index
    if types == 'cpu':
        prods = products_cpu.iloc[index]
    else:
        prods = products_gpu.iloc[index]
        
    resul = []
    for i, row in prods.iterrows():
        if i != idx:
            print(row)
            resul.append(uuid.UUID(i))
    return resul

In [32]:
def get_recommendations(df_wishlist, types):
        recs = []
        
        
        wishlist = list(df_wishlist['product_id'])
    
        for i, product in enumerate(wishlist):
            
            if types == 'cpu':
                response = list(recommend(product, similarity_cpu, types))
            else:
                response = list(recommend(product, similarity_gpu, types))
            
            recs.append(response)
        print(recs)
        return recs
 

In [33]:
   
cpus = get_recommendations(wishlist_cpu, 'cpu')
gpus = get_recommendations(wishlist_gpu, 'gpu')

result_cpu = pd.DataFrame(wishlist_cpu['product_id'], columns=['id','recommendations'])
result_gpu = pd.DataFrame(wishlist_gpu['product_id'], columns=['id','recommendations'])

wishlist_gpu['recommendations'] = gpus
wishlist_cpu['recommendations'] = cpus

wishlist_cpu.set_index('product_id', inplace=True)
wishlist_gpu.set_index('product_id', inplace=True)

# return wishlist_gpu, wishlist_cpu
wishlist_cpu

brand                                                           AMD
retailer                                                    Evetech
type                                                            cpu
description_cl    system memory type-ddr4  pci express® version-...
Name: 1a4c23c6-53d9-4728-bc8e-1723a5d0511e, dtype: object
brand                                                           AMD
retailer                                                    Evetech
type                                                            cpu
description_cl    system memory type-ddr4  pci express® version-...
Name: 1653365c-ffb9-4b8f-b96a-42a5b4817dde, dtype: object
brand                                                           AMD
retailer                                                    Evetech
type                                                            cpu
description_cl    system memory type-ddr4  pci express® version-...
Name: d0b620a6-8078-4f71-a95f-1bdc7fae7d91, dtype: object
brand     

<ipython-input-33-81b2a41c680f>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wishlist_gpu['recommendations'] = gpus
<ipython-input-33-81b2a41c680f>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wishlist_cpu['recommendations'] = cpus


,type,recommendations
product_id,,
04cd4be7-b116-4a05-8a70-99402fd984f8,cpu,"[1a4c23c6-53d9-4728-bc8e-1723a5d0511e, 1653365..."


In [34]:
wishlist_gpu

,type,recommendations
product_id,,
6c06f47c-c4ee-4cd3-88a9-bb55890aa923,gpu,"[34748fb8-6c65-4bfe-9d8f-3d8749b18cbf, 79cc0dc..."
34748fb8-6c65-4bfe-9d8f-3d8749b18cbf,gpu,"[04eff54f-09c1-4f43-b72b-a86392820762, 1b01afa..."
79cc0dc7-383f-43fe-8a9d-406012f6edb7,gpu,"[6c06f47c-c4ee-4cd3-88a9-bb55890aa923, 34748fb..."
909f4df4-dd36-46fb-bc39-c5208598b5ec,gpu,"[6c06f47c-c4ee-4cd3-88a9-bb55890aa923, 34748fb..."
b7dc3042-0330-42d9-bebb-59601adfb987,gpu,"[6c06f47c-c4ee-4cd3-88a9-bb55890aa923, 34748fb..."


In [36]:
def update_rec_table(recs, types):
    new_products = []
    if types == "cpu":
        table_name = "recommendation_cpu"
    else:
        table_name = "recommendation_gpu"

    try:

        for i, row in recs.iterrows():
            query = sql.SQL("SELECT product_id FROM {} WHERE product_id = %s").format(sql.Identifier(table_name))
                     
            # query = ("""SELECT product_id FROM recommendation_gpu WHERE product_id = %s""")
            cur.execute(query, (i,))
            product = cur.fetchone()
            
            if product is not None:
                print('NotNone')
                for recs in row['recommendations']:
                    query = sql.SQL(" UPDATE {} SET recommended_id = %s WHERE product_id = %s").format(sql.Identifier(table_name))
                    cur.execute(query, (recs, uuid.UUID(i)))
                    con.commit()
            else:
                new_products.append((i, row['recommendations']))
                
            
    except(psycopg2.DatabaseError) as err:
        print(err)
    finally:
        return new_products


IndentationError: unexpected indent (<ipython-input-36-23e787036ea3>, line 19)

In [22]:
new_gpu = update_rec_table(wishlist_gpu, 'gpu')
new_cpu = update_rec_table(wishlist_cpu, 'cpu')

none?
none?
none?
none?
none?
None
none!!


In [23]:
new_gpu

[('6c06f47c-c4ee-4cd3-88a9-bb55890aa923',
  [UUID('6c06f47c-c4ee-4cd3-88a9-bb55890aa923'),
   UUID('34748fb8-6c65-4bfe-9d8f-3d8749b18cbf'),
   UUID('79cc0dc7-383f-43fe-8a9d-406012f6edb7'),
   UUID('d00c8bb0-5db9-4caa-91a2-c9963eee8a3f'),
   UUID('b7dc3042-0330-42d9-bebb-59601adfb987'),
   UUID('f5e646d6-1019-4f35-b8b6-ff9c40b73c5c')]),
 ('34748fb8-6c65-4bfe-9d8f-3d8749b18cbf',
  [UUID('04eff54f-09c1-4f43-b72b-a86392820762'),
   UUID('1b01afaf-e184-4f70-8ad0-7e85280fc980'),
   UUID('909f4df4-dd36-46fb-bc39-c5208598b5ec'),
   UUID('1476521c-0f79-4757-b36d-69a4449b8781'),
   UUID('f93ad309-e2dc-43ed-b94e-df9ee8ded9c8'),
   UUID('bbf1d6b6-be3f-4c1d-94b3-b0423f307dbf')]),
 ('79cc0dc7-383f-43fe-8a9d-406012f6edb7',
  [UUID('6c06f47c-c4ee-4cd3-88a9-bb55890aa923'),
   UUID('34748fb8-6c65-4bfe-9d8f-3d8749b18cbf'),
   UUID('79cc0dc7-383f-43fe-8a9d-406012f6edb7'),
   UUID('d00c8bb0-5db9-4caa-91a2-c9963eee8a3f'),
   UUID('b7dc3042-0330-42d9-bebb-59601adfb987'),
   UUID('f5e646d6-1019-4f35-b8b6-ff9c

In [24]:
new_cpu

[('04cd4be7-b116-4a05-8a70-99402fd984f8',
  [UUID('04cd4be7-b116-4a05-8a70-99402fd984f8'),
   UUID('1a4c23c6-53d9-4728-bc8e-1723a5d0511e'),
   UUID('1653365c-ffb9-4b8f-b96a-42a5b4817dde'),
   UUID('d0b620a6-8078-4f71-a95f-1bdc7fae7d91'),
   UUID('18b0243a-ff92-47b0-bbbd-eef96f55ccee'),
   UUID('b8a06283-c102-4e7b-becf-fd7ddb587209')])]

In [25]:
def insert_cpu(id, cpu_arr):
    psycopg2.extras.register_uuid()
    for cpu in cpu_arr:
        query = ("""INSERT INTO recommendation_cpu (product_id, recommended_id) VALUES (%s, %s);""")
        cur.execute(query,(id, cpu))
        con.commit()


def insert_gpu(id, gpu_arr):
    psycopg2.extras.register_uuid()
    print(gpu_arr)
    for gpu in gpu_arr:
        query = ("""INSERT INTO recommendation_gpu (product_id, recommended_id) VALUES (%s, %s);""")
        cur.execute(query,(id, gpu))
        con.commit()


In [26]:
if new_gpu is not None:
    df_gpu = pd.DataFrame(new_gpu, columns=['id', 'recs'])
    for i, row in df_gpu.iterrows():
        insert_gpu(row['id'], row['recs'])

if new_cpu is not None:
    df_cpu = pd.DataFrame(new_cpu, columns=['id', 'recs'])
    for i, row in df_cpu.iterrows():
        insert_cpu(row['id'], row['recs'])

con.close()

[UUID('6c06f47c-c4ee-4cd3-88a9-bb55890aa923'), UUID('34748fb8-6c65-4bfe-9d8f-3d8749b18cbf'), UUID('79cc0dc7-383f-43fe-8a9d-406012f6edb7'), UUID('d00c8bb0-5db9-4caa-91a2-c9963eee8a3f'), UUID('b7dc3042-0330-42d9-bebb-59601adfb987'), UUID('f5e646d6-1019-4f35-b8b6-ff9c40b73c5c')]
[UUID('04eff54f-09c1-4f43-b72b-a86392820762'), UUID('1b01afaf-e184-4f70-8ad0-7e85280fc980'), UUID('909f4df4-dd36-46fb-bc39-c5208598b5ec'), UUID('1476521c-0f79-4757-b36d-69a4449b8781'), UUID('f93ad309-e2dc-43ed-b94e-df9ee8ded9c8'), UUID('bbf1d6b6-be3f-4c1d-94b3-b0423f307dbf')]
[UUID('6c06f47c-c4ee-4cd3-88a9-bb55890aa923'), UUID('34748fb8-6c65-4bfe-9d8f-3d8749b18cbf'), UUID('79cc0dc7-383f-43fe-8a9d-406012f6edb7'), UUID('d00c8bb0-5db9-4caa-91a2-c9963eee8a3f'), UUID('b7dc3042-0330-42d9-bebb-59601adfb987'), UUID('f5e646d6-1019-4f35-b8b6-ff9c40b73c5c')]
[UUID('6c06f47c-c4ee-4cd3-88a9-bb55890aa923'), UUID('34748fb8-6c65-4bfe-9d8f-3d8749b18cbf'), UUID('79cc0dc7-383f-43fe-8a9d-406012f6edb7'), UUID('d00c8bb0-5db9-4caa-91a2-